In [1]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator
    
import numpy as np
import scipy as sp
from scipy import constants
from pylab import *
import joblib


import os,sys
from importlib import reload
import copy
sys.path.append("../tracker")

In [3]:
import kalmanfilter as KF
import utilities as Util
import trackfinder as TF
import datatypes

reload(TF)
reload(Util)

<module 'utilities' from '../tracker\\utilities.py'>

In [4]:
tracks=joblib.load("tracks_example.joblib")
# tracks

In [ ]:
class chi2_vertex:
    def __init__(self, tracks):
        self.tracks=tracks
        self.func_code = iminuit.util.make_func_code(['x0', 'y0', 'z0', 't0'])
    def __call__(self, x0, y0, z0, t0):
        error=0
        point = [x0, y0, z0, t0]
        for track in self.tracks:
            error += Util.vertex.chi2_point_track(point, track)
        return error        


    
def fit_vertex(tracks, guess):
    x0_init, y0_init, z0_init,t0_init = guess

    m = Minuit(chi2_vertex(tracks),x0=x0_init, y0=y0_init, z0=z0_init, t0=t0_init)
    # m.fixed["y0"]=True
    m.limits["x0"]=(-100000,100000)
    m.limits["y0"]=(-100000,100000)
    m.limits["z0"]=(-100000,100000)
    m.limits["t0"]=(-100,1e5)
    m.errors["x0"]=0.1
    m.errors["y0"]=0.1
    m.errors["z0"]=0.1
    m.errors["t0"]=0.1

    m.migrad()  # run optimiser
    m.hesse()   # run covariance estimator
    return m

class VertexFitter:
    def init_state(self, track_pair):
        return

    def filter